# Non linear optimization: linesearch

## Introduction to optimization and operations research

Michel Bierlaire


In [ ]:

from collections.abc import Callable

import numpy as np



# Question 1
Consider the function  $f:\mathbb{R}^2 \rightarrow \mathbb{R}$
defined as
$$
f(x_1, x_2) = x_1^2 + 2x_2^2,
$$
Implement a Python function that calculates its value, gradient and second derivative matrix.

In [ ]:
def my_function(x: np.array) -> tuple[float, np.array, np.array]:
    """Function and its derivatives

    :param x: array of dimension 2
    :return: a tuple with the value of the function, the gradient and the hessian
    """
    f = x[0] * x[0] + 2 * x[1] * x[1]
    gradient = np.array([[2 * x[0], 4 * x[1]]])
    hessian = np.array([[2, 0], [0, 4]])
    return f, gradient, hessian



Test the function at the point $(9, 1).$

In [ ]:
x = np.array([9, 1])
function, gradient, hessian = my_function(x)
print(f'f(x)={function}')
print(f'gradient(x)={gradient}')
print(f'hessian(x)=\n{hessian}')


# Question 1
Implement the two Wolfe conditions.

Wolfe conditions

In [ ]:
def wolfe_conditions(
    current_point: np.array,
    direction: np.array,
    alpha: float,
    a_function: Callable[[np.array], tuple[float, np.array, np.array]],
    beta_1,
    beta_2,
) -> tuple[bool, bool]:
    """
    First wolfe condition

    :param current_point: current iterate
    :param direction: direction to follow
    :param alpha: step along the direction
    :param a_function: function being minimized
    :param beta_1: parameter of the first Wolfe condition
    :param beta_2: parameter of the second Wolfe condition
    :return: two boolean corresponding to the two conditions.
    """
    current_function, current_gradient, _ = a_function(current_point)
    directional_derivative = np.dot(
        current_gradient, direction
    )
    if directional_derivative >= 0:
        error_msg = (
            f'The direction must be a descent direction. The directional derivative is non negative: '
            f'{directional_derivative}'
        )
    new_iterate = current_point + alpha * direction
    new_function, new_gradient, _ = a_function(new_iterate)
    first_wolfe_verified = (
        new_function <= current_function + alpha * beta_1 * directional_derivative
    )
    second_wolfe_verified = (
        np.dot(new_gradient, direction) >= beta_2 * directional_derivative
    )
    return first_wolfe_verified, second_wolfe_verified



Consider the point $x^0 = (9, 1)$, and the direction
$$
d = \left(\begin{array}{c}-18 \\ -4\end{array}\right).
$$
Test the Wolfe conditions with $\alpha_0=0.05$,
$\beta_1=0.01$ and $\beta_2=0.8$.

In [ ]:

x_0 = np.array([9, 1])
direction = np.array([-18, -4])
alpha = 0.05
beta_1 = 0.01
beta_2 = 0.8
wolfe_1, wolfe_2 = wolfe_conditions(
    current_point=x_0,
    direction=direction,
    alpha=alpha,
    a_function=my_function,
    beta_1=beta_1,
    beta_2=beta_2,
)
print(f'Wolfe 1 verified? {wolfe_1}')
print(f'Wolfe 2 verified? {wolfe_2}')



# Question 2
Implement the line search algorithm to find a step along a direction.
At each iteration, print the current value of $\alpha$, $\alpha_\ell$ and $\alpha_r$, as well as the
condition which is violated.

In [ ]:
def linesearh(
    current_point: np.array,
    direction: np.array,
    a_function: Callable[[np.array], tuple[float, np.array, np.array]],
    alpha_0: float,
    beta_1: float,
    beta_2: float,
    expansion_lambda: float,
) -> float:
    """Implementation of the line search algorithm (p. 274)

    [Bierlaire (2018)](https://transp-or.epfl.ch/books/optimization/html/OptimizationPrinciplesAlgorithms2018.pdf)

    :param current_point: current iterate
    :param direction: direction to follow
    :param a_function: function being minimized
    :param alpha_0: initial value for the step
    :param beta_1: parameter of the first Wolfe condition
    :param beta_2: parameter of the second Wolfe condition
    :param expansion_lambda: expansion factor
    :return: step verifying both Wolfe conditions.
    """
    if beta_1 >= beta_2:
        error_message = f'The value of {beta_1=} and {beta_2} are incompatible.'
        raise ValueError(error_message)

    alpha = alpha_0
    alpha_left = 0
    alpha_right = np.inf

    both_conditions_are_verified = False

    while not both_conditions_are_verified:
        print(f'{alpha=:.3g} {alpha_left=:.3g} {alpha_right=:.3g} ', end='')

        first_wolfe, second_wolfe = wolfe_conditions(
            current_point,
            direction,
            alpha,
            a_function,
            beta_1,
            beta_2,
        )
        if not first_wolfe:
            # The step is too long
            alpha_right = alpha
            alpha = (alpha_left + alpha_right) / 2.0
            violated = 'W1'
        elif not second_wolfe:
            # The step is too short
            alpha_left = alpha
            if alpha_right == np.inf:
                alpha *= expansion_lambda
            else:
                alpha = (alpha_left + alpha_right) / 2.0
            violated = 'W2'
        else:
            both_conditions_are_verified = True
            violated = '--'
        print(violated)
    return alpha



Apply the line search algorithm with $\alpha_0=0.05$,
$\beta_1=0.01$, $\beta_2=0.8$ and $\lambda=20$.

In [ ]:
alpha_star = linesearh(
    current_point=x_0,
    direction=direction,
    a_function=my_function,
    alpha_0=0.05,
    beta_1=beta_1,
    beta_2=beta_2,
    expansion_lambda=20,
)


In [ ]:
print(f'{alpha_star=:.3g}')